### Melon100 Chart 
* 100곡의 노래의 제목과 SongID 추출해서 list에 저장하기
* 100곡 노래의 상세정보를 추출해서 list와 dict에 저장해서 json 파일로 저장하기
* json 파일을 load하여 Pandas의 DataFrame에 저장하기
* DataFrame 객체를 DB의 Table에 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [28]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    #<div id="tb_list"><tr><a>
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))   
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")
    song_list = []
    for idx,a_tag in enumerate(a_tags,1):
        song_dict = {}
        #<a href="">노래제목</a>
        song_title = a_tag.text
        song_dict['song_title'] = song_title
        
        href_value = a_tag['href']
        #Song ID를 찾기 위한 정규표현식
        matched = re.search(r'(\d+)\);', href_value)
        if matched:
            song_id = matched.group(1)
            song_dict['song_id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['song_detail_url'] = song_detail_url
            #print(song_dict)
            song_list.append(song_dict)

print(len(song_list))            

200
100
100


In [45]:
import requests
from bs4 import BeautifulSoup
import re

req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list[:1],1):
    #노래1곡의 상세정보를 저장할 dict 
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']
    
    res = requests.get(song_detail_url, headers=req_header_dict)    
    print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        song_detail_dict['곡명'] = song['song_title']
        # [<span>임영웅</span>, <span></span>]
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text
        
        print(soup.select("div.meta dd"))
        song_dd = soup.select("div.meta dd")
        if song_dd:
            #print(song_dd[0].text)
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text
        print(song_detail_dict)
        song_detail_list.append(song_detail_dict)

200
[<dd><a href="javascript:melon.link.goAlbumDetail('10735654');">신사와 아가씨 OST Part.2</a></dd>, <dd>2021.10.11</dd>, <dd>발라드, 국내드라마</dd>, <dd>Flac 16/24bit</dd>]
{'곡명': '사랑은 늘 도망가', '가수': '임영웅', '앨범': '신사와 아가씨 OST Part.2'}
200
[<dd><a href="javascript:melon.link.goAlbumDetail('10816959');">취중고백</a></dd>, <dd>2021.12.19</dd>, <dd>발라드</dd>, <dd>Flac 16bit</dd>]
{'곡명': '취중고백', '가수': '김민석 (멜로망스)', '앨범': '취중고백'}
